In [1]:
import json
# Open and read the JSON file
with open("2_504_summaries_file_name_chunk.json", "r", encoding="utf-8") as file:
    data = json.load(file)  # Parse JSON into a Python dictionary


In [36]:
SUMMARIZATION_PROMPT = """
You are an AI specializing in text analysis and summarization. Your task is to give me the main content information that the current chunk is processing (Main topic and information being processed) from the input filename, the content being processed from previous chunks and the current chunk.

1. Output requirements:
You must ensure the following:

Main topic and core content.
The overview of the topic does not need to be too detailed. But it is necessary to extract important words to represent the topic being processed
Summarize previously mentioned content to provide necessary context.
Concise and focused content to summarize the context
Update or expand the summary if necessary, avoiding irrelevant or far-fetched information.
Output is in Vietnamese.

2. Input format:
Filename: {file_name}
Previous summary: {previous_summary}
Current chunk: {input_chunk}

3. Output format:
Document references to [Topic], [Overview of that topic based on previous information]

4. Key considerations:
Extract only the information currently being processed in the chunk.
Avoid including content that is too far-fetched or irrelevant.

5. Output:
"""

In [37]:
from openai import AzureOpenAI
from dotenv import load_dotenv
import os

load_dotenv()

api_base = os.getenv("GPT4O__ENDPOINT")
api_key = os.getenv("GPT4O__KEY")
deployment_name ="gpt-4o-mini"
api_version = os.getenv("GPT4O__API_VERSION")

client = AzureOpenAI(
    api_key=api_key,
    api_version=api_version,
    base_url=f"{api_base}/openai/deployments/{deployment_name}"
)

In [38]:
print(data['chunks'])

[' \nQUY TRÌNH\nXử lý sự cố \n| ISO/IEC 27001:2013& ISO 9001:2015 | ISO/IEC 27001:2013& ISO 9001:2015 |\n| ----- | ----- |\n| Mã hiệu                           | 2-504                             |\n| Ngày hiệu lực                     | 2023/11/09                        |\n| Lần ban hành                      | 01                                | \nBảng đầu tiên cung cấp thông tin về quy trình xử lý sự cố, bao gồm mã hiệu, ngày hiệu lực và lần ban hành, nhằm xác định rõ ràng các tiêu chuẩn áp dụng cho quy trình này.\nLỊCH SỬ SỬA ĐỔI TÀI LIỆU \n| STT | Mục sửa đổi | Nội dung sửa đổi | Ngày sửa đổi | Phiên bản sửa đổi |\n| ----- | ----- | ----- | ----- | ----- |\n| 1   | Toàn bộ     | Ban hành         | 2023/11/09   | 01                | \nBảng thứ hai ghi lại lịch sử sửa đổi tài liệu, cho thấy các thay đổi đã được thực hiện, ngày sửa đổi và phiên bản, giúp theo dõi sự phát triển và cập nhật của quy trình.\nMỤC LỤC I. MỤC ĐÍCH ..............................................................

In [39]:
summaries = ["This is first paragraph in files."]
for chunk in data["chunks"]:
    response = client.chat.completions.create(
        model=deployment_name,
        messages=[
            {"role": "system", "content": "You are an AI specializing in text analysis and summarization. Your task is to give me the main content information that the current chunk is processing (Main topic and information being processed) from the input filename, the content being processed from previous chunks and the current chunk."},
            {"role": "user", "content": SUMMARIZATION_PROMPT.format(file_name="1-202 Nội Quy Công Ty", input_chunk = chunk, previous_summary = summaries[-1])},
        ],
        temperature=0.0,
    )
    summaries.append(response.choices[0].message.content)

In [40]:
with open("main_topic.txt", "a", encoding="utf-8") as file:  # Open file in append mode
    for sum in summaries:
        file.write("Context:" + sum + "\n")
        file.write("=====================================\n")  # Add a separator after each summary

In [18]:
## Version 3
with open("summaries_1_200.txt", "a", encoding="utf-8") as file:  # Open file in append mode
    for index, chunk in enumerate(data["chunks"]):
        file.write("Context:" + summaries[index] + "\n")  # Append each summary followed by a newline
        file.write("Chunk: " + str(chunk) + "\n")
        file.write("=====================================\n")  # Add a separator after each summary

## Documents Summary

In [46]:
DOCUMENT_SUMMARY = """
Summarize all sub-summaries of the document to create a comprehensive summary. Ensure that the summary fully reflects the main topic and core content of the document. Provide necessary context by summarizing previously mentioned content while highlighting the latest information from later sections. Avoid adding irrelevant information or making overly speculative interpretations.

Input Format:
Filename: {file_name}
List of sub-summaries: {list_of_sub_summaries}

Requirements: 
Output is Vietnamese. 

Desired Output Format:
Document Topic: [Main topic]
Document Summeries: [Comprehensive summary of key information]
"""

In [47]:
response = client.chat.completions.create(
        model=deployment_name,
        messages=[
            {"role": "system", "content": "Summarize all sub-summaries of the document to create a comprehensive summary. Ensure that the summary fully reflects the main topic and core content of the document. Provide necessary context by summarizing previously mentioned content while highlighting the latest information from later sections. Avoid adding irrelevant information or making overly speculative interpretations."},
            {"role": "user", "content": DOCUMENT_SUMMARY.format(file_name="1-202 Nội Quy Công Ty", list_of_sub_summaries = str(summaries))},
        ],
        temperature=0.0,
    )

In [48]:
print(response.choices[0].message.content)

Document Topic: Nội Quy Công Ty

Document Summaries: Tài liệu này trình bày quy định và quy trình xử lý sự cố tại Công ty TNHH Sun Asterisk Việt Nam, nhằm giảm thiểu tác động tiêu cực đến hoạt động và kinh doanh của công ty. Quy trình bao gồm các bước từ phát hiện, báo cáo, phân tích đến khắc phục sự cố, với mục tiêu cải tiến chất lượng và an ninh thông tin. Sự cố được định nghĩa là những lỗi ảnh hưởng đến hoạt động, dịch vụ và uy tín của công ty, cần được can thiệp kịp thời để khôi phục hoạt động bình thường. 

Tài liệu cũng đề cập đến thiệt hại tài chính do sự cố gây ra và yêu cầu chứng cứ liên quan phải đảm bảo tính toàn vẹn và chính xác. Các sự cố được phân loại theo mức độ ưu tiên và nghiêm trọng, với các tiêu chí rõ ràng để xác định cách thức xử lý phù hợp. Quy trình phát hiện và báo cáo sự cố nhấn mạnh tầm quan trọng của việc thông báo kịp thời và thu thập chứng cứ, trong khi hành động khắc phục tức thời được phân loại theo mức độ ưu tiên.

Sau khi sự cố được xử lý, quy trình ph

In [49]:
doc_sum = response.choices[0].message.content

## List Global context content

In [51]:
lst_global_context = []
for sum in summaries:
    lst_global_context.append("Documents Summary: \n " + doc_sum + "\n" + "Content Previous: \n" + sum)
    
## Version 3
with open("global_context.txt", "a", encoding="utf-8") as file:  # Open file in append mode
    for index, context in enumerate(lst_global_context):
        file.write("Context:" + context + "\n")  # Append each summary followed by a newline
        file.write("=====================================\n")  # Add a separator after each summary